<a href="https://colab.research.google.com/github/researcheraman/Microbial_analyzer/blob/main/Workingscratchpad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# -----------------------------
# Enhanced Analyzer: All Classes + Host-Specific Prions/Viroids + Multi-Source Images
# FIXED: AttributeError, sequence retrieval failures, deprecated pairwise2, feature gating
# -----------------------------
!pip install biopython
from ipywidgets import widgets, VBox, HBox, Button, Output, Dropdown
from IPython.display import display, clear_output, Image, Markdown, HTML
from Bio import Entrez, SeqIO
from Bio.Blast import NCBIWWW, NCBIXML
from Bio.SeqRecord import SeqRecord
from Bio.Align import PairwiseAligner  # Modern replacement for pairwise2
from Bio.Phylo.TreeConstruction import DistanceTreeConstructor, _DistanceMatrix
from Bio import Phylo
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time
import requests
from bs4 import BeautifulSoup
import re

# ----------- Set your NCBI email -----------
Entrez.email = "jannatisback002@gmail.com"  # Replace with your email

# Fallback image URLs
FALLBACK_IMAGES = {
    "Escherichia coli": "https://upload.wikimedia.org/wikipedia/commons/thumb/0/0b/EscherichiaColi_NIAID.jpg/800px-EscherichiaColi_NIAID.jpg",
    "Lactobacillus acidophilus": "https://upload.wikimedia.org/wikipedia/commons/thumb/5/5a/Lactobacillus_acidophilus.jpg/800px-Lactobacillus_acidophilus.jpg",
    "SARS-CoV-2": "https://upload.wikimedia.org/wikipedia/commons/7/76/SARS-CoV-2_virus.jpg",
    "HIV": "https://upload.wikimedia.org/wikipedia/commons/7/7e/HIV-budding-Color.jpg"
}

# --- 1. Widgets ---
species_input = widgets.Text(
    description="Species (comma separated):",
    placeholder="Escherichia coli, SARS-CoV-2, Plasmodium falciparum, human prion, potato spindle tuber viroid"
)

gene_dropdown = widgets.Dropdown(
    options=["16S rRNA", "23S rRNA", "recA", "rpoB", "gyrB", "atpD", "tufA", "infB"],
    value="16S rRNA",
    description="Gene/RNA:"
)

feature_dropdown = widgets.Dropdown(
    options=[
        "All Features",
        "DNA/RNA Sequence Only",
        "Protein Sequence Only",
        "BLAST Comparisons (DNA/RNA)",
        "BLAST Comparisons (Protein)",
        "Phylogenetic Tree (DNA/RNA)",
        "Phylogenetic Tree (Protein)",
        "Repeat Analysis",
        "Taxonomy & Images Only",
        "Colored Alignments"
    ],
    value="All Features",
    description="Select Features:"
)

tree_method_dropdown = widgets.Dropdown(
    options=["UPGMA", "Neighbor-Joining"],
    value="UPGMA",
    description="Tree method:"
)

run_button = Button(description="Run Analysis", button_style='success')

output = Output()
display(VBox([species_input, gene_dropdown, feature_dropdown, tree_method_dropdown, run_button, output]))

# --- 2. Helper functions ---

def fetch_dna_sequences(species_list, gene_name):
    sequences = {}
    notes = {}
    for species in species_list:
        species = species.strip()
        original = species

        # Prion special handling
        if "prion" in species.lower():
            query = f'"{original}"[All Fields] AND "PRNP"[Gene] AND 100:5000[SLEN]'
            note = "Prions have no DNA/RNA genome. Showing DNA sequence of the host PRNP gene that encodes the prion protein."
        # Viroid special handling
        elif "viroid" in species.lower():
            query = f'"{original}"[All Fields] OR "viroid"[Organism] AND 100:1000[SLEN]'
            note = "Viroids are non-coding circular RNA. Showing RNA sequence (converted to DNA letters for analysis)."
        # Default (broadened for viruses)
        else:
            query = f'("{original}"[Organism] OR "{original}"[Title]) AND ("{gene_name}"[Gene] OR "{gene_name}"[All Fields]) AND 100:50000[SLEN]'
            note = None

        try:
            handle = Entrez.esearch(db="nucleotide", term=query, retmax=3)
            ids = Entrez.read(handle)["IdList"]
            handle.close()
            if ids:
                time.sleep(0.35)
                fetch_handle = Entrez.efetch(db="nucleotide", id=ids[0], rettype="fasta", retmode="text")
                seq_record = SeqIO.read(fetch_handle, "fasta")
                fetch_handle.close()
                seq_record.id = original
                seq_record.description = ""
                seq_record.seq = seq_record.seq.upper().replace("U", "T")  # RNA to cDNA conversion
                sequences[original] = seq_record
                notes[original] = note
            else:
                # RNA fallback
                rna_query = f'("{original}"[Organism] OR "{original}"[Title]) AND biomol_rna[PROP] AND 100:50000[SLEN]'
                handle = Entrez.esearch(db="nucleotide", term=rna_query, retmax=3)
                ids = Entrez.read(handle)["IdList"]
                handle.close()
                if ids:
                    time.sleep(0.35)
                    fetch_handle = Entrez.efetch(db="nucleotide", id=ids[0], rettype="fasta", retmode="text")
                    seq_record = SeqIO.read(fetch_handle, "fasta")
                    fetch_handle.close()
                    seq_record.id = original
                    seq_record.seq = seq_record.seq.upper().replace("U", "T")
                    sequences[original] = seq_record
                    notes[original] = "No DNA sequence found – This is an RNA organism. Showing RNA genome (converted to cDNA format for analysis)."
                else:
                    sequences[original] = None
                    notes[original] = f"No sequence found for '{original}' in NCBI."
        except Exception as e:
            sequences[original] = None
            notes[original] = f"Error fetching '{original}': {str(e)}"

    return sequences, notes

def fetch_protein_sequences(species_list):
    sequences = {}
    notes = {}
    for species in species_list:
        query = f'"{species}"[Organism]'
        note = None
        if "prion" in species.lower():
            query = f'"{species}"[All Fields] AND "prion protein"[Gene]'
            note = "Prions have no genome. Showing host PRNP protein sequence."
        try:
            handle = Entrez.esearch(db="protein", term=query, retmax=1)
            ids = Entrez.read(handle)["IdList"]
            handle.close()
            if ids:
                time.sleep(0.35)
                fetch_handle = Entrez.efetch(db="protein", id=ids[0], rettype="fasta", retmode="text")
                seq_record = SeqIO.read(fetch_handle, "fasta")
                fetch_handle.close()
                seq_record.id = species
                seq_record.description = ""
                sequences[species] = seq_record
                notes[species] = note
            else:
                sequences[species] = None
                notes[species] = f"No protein found for '{species}' in NCBI."
        except Exception as e:
            sequences[species] = None
            notes[species] = f"Error: {e}"
    return sequences, notes

def perform_blast_comparisons(seq_dict, seq_type="DNA", max_hits=5):
    comparisons = []
    program = "blastn" if seq_type == "DNA" else "blastp"
    database = "nt" if seq_type == "DNA" else "nr"
    for ref_name, ref_record in seq_dict.items():
        if ref_record is None:
            continue
        ref_seq = str(ref_record.seq)
        try:
            result_handle = NCBIWWW.qblast(program, database, ref_seq, hitlist_size=max_hits)
            blast_records = NCBIXML.parse(result_handle)
            for blast_record in blast_records:
                for alignment in blast_record.alignments:
                    hit_title = alignment.title.split()[0]
                    for hsp in alignment.hsps[:1]:
                        identity = (hsp.identities / hsp.align_length) * 100
                        comparisons.append({
                            f"{ref_name} vs {hit_title}": f"{identity:.2f}%",
                            "E-value": f"{hsp.expect:.2e}",
                            "Bit Score": hsp.bits,
                            "Alignment Length": hsp.align_length,
                            "Query Coverage": f"{hsp.align_length / len(ref_seq) * 100:.1f}%"
                        })
            result_handle.close()
            time.sleep(1)
        except Exception as e:
            print(f"❌ BLAST error for {ref_name}: {e}")
    if comparisons:
        df = pd.DataFrame(comparisons)
        return df.sort_values("Bit Score", ascending=False)
    return pd.DataFrame()

def analyze_dna(seq_record):
    seq = str(seq_record.seq)
    length = len(seq)
    gc = 100 * (seq.count('G') + seq.count('C')) / length if length > 0 else 0
    return {"Length (bp)": length, "GC Content (%)": f"{gc:.2f}"}

def analyze_protein(seq_record):
    seq = str(seq_record.seq)
    return {"Length (aa)": len(seq)}

def compute_distance_matrix(seq_dict):
    valid_records = [r for r in seq_dict.values() if r and len(r.seq) > 0]
    if len(valid_records) < 2:
        raise ValueError("Need at least 2 sequences")
    n = len(valid_records)
    names = [r.id for r in valid_records]
    dist_matrix = np.zeros((n, n))
    aligner = PairwiseAligner()
    for i in range(n):
        for j in range(i + 1, n):
            alignments = aligner.align(str(valid_records[i].seq), str(valid_records[j].seq))
            score = alignments[0].score
            identity = score / max(len(str(valid_records[i].seq)), len(str(valid_records[j].seq)))
            dist = 1 - identity
            dist_matrix[i, j] = dist
            dist_matrix[j, i] = dist
    lower_triangle = [[dist_matrix[i, j] for j in range(i + 1)] for i in range(n)]
    return names, lower_triangle

def build_and_plot_tree(names, matrix, method, title):
    dm = _DistanceMatrix(names, matrix)
    constructor = DistanceTreeConstructor()
    tree = constructor.upgma(dm) if method == "UPGMA" else constructor.nj(dm)
    plt.figure(figsize=(12, 8))
    Phylo.draw(tree, do_show=False)
    plt.title(f"{title} [Data from NCBI/GenBank]")
    plt.tight_layout()
    plt.show()

def fetch_taxonomy(species):
    try:
        handle = Entrez.esearch(db="taxonomy", term=species)
        taxid_list = Entrez.read(handle)["IdList"]
        handle.close()
        if taxid_list:
            time.sleep(0.3)
            taxid = taxid_list[0]
            handle = Entrez.efetch(db="taxonomy", id=taxid, retmode="xml")
            tax_record = Entrez.read(handle)[0]
            handle.close()
            lineage = tax_record.get("Lineage", "No lineage")
            return f"{lineage} [NCBI TaxID: {taxid}]"
    except:
        return "Error fetching taxonomy"
    return "Not found"

def fetch_google_image(species):
    # Wikipedia first
    try:
        search_term = species.replace(" ", "_")
        url = f"https://en.wikipedia.org/w/api.php?action=query&format=json&list=search&srsearch={search_term}&srlimit=1"
        response = requests.get(url).json()
        if response["query"]["search"]:
            page_title = response["query"]["search"][0]["title"].replace(" ", "_")
            img_url = f"https://en.wikipedia.org/w/api.php?action=query&format=json&prop=pageimages&piprop=original&titles={page_title}"
            img_response = requests.get(img_url).json()
            pages = img_response["query"]["pages"]
            for page_id in pages:
                if "original" in pages[page_id]:
                    return pages[page_id]["original"]["source"]
    except:
        pass

    # Google fallback
    try:
        headers = {"User-Agent": "Mozilla/5.0"}
        params = {"q": species, "tbm": "isch", "hl": "en"}
        response = requests.get("https://www.google.com/search", params=params, headers=headers, timeout=10)
        soup = BeautifulSoup(response.text, "html.parser")
        imgs = soup.find_all("img")
        for img in imgs:
            src = img.get("src")
            if src and "http" in src and "google" not in src.lower() and "gstatic" not in src:
                return src
        scripts = soup.find_all("script")
        for script in scripts:
            if script.string and "data:image" not in script.string:
                match = re.search(r'[](https://[^"]+\.jpg[^"]*)"', script.string)
                if match:
                    return match.group(1)
    except:
        pass

    return None

def display_fasta(seq_record, title):
    fasta_str = f">{seq_record.id}\n{str(seq_record.seq)}"
    display(Markdown(f"### {title}\n```fasta\n{fasta_str}\n```"))

def find_tandem_repeats(seq, min_length=3):
    repeats = []
    i = 0
    n = len(seq)
    while i < n - min_length + 1:
        if i + 1 < n and seq[i] == seq[i + 1]:
            start = i
            char = seq[i]
            while i < n and seq[i] == char:
                i += 1
            length = i - start
            if length >= min_length:
                repeats.append((char, start + 1, i, length))
        else:
            i += 1
    return repeats

def display_colored_seq(seq_record, repeats, motif_counts, total_species):
    seq = str(seq_record.seq)
    html_parts = [f"<h4>{seq_record.id} (Colored Repeats)</h4><pre style='font-family: monospace; white-space: pre;'>"]
    i = 0
    line_length = 60
    while i < len(seq):
        matched_repeat = None
        for r in repeats:
            if i + 1 == r[1]:
                char, start, end, run_len = r
                run_str = seq[i:i + run_len]
                count = motif_counts.get((char, run_len), 1)
                share_frac = count / total_species
                if share_frac == 1.0:
                    color_style = "background-color: lime; color: black; font-weight: bold;"
                elif share_frac >= 2 / total_species:
                    color_style = "background-color: lightgreen; color: darkgreen;"
                else:
                    color_style = "background-color: yellow; color: red;"
                title = f"{char} x {run_len} at pos {start}-{end-1} (Shared in {count}/{total_species} species)"
                html_parts.append(f'<span style="{color_style}" title="{title}">{run_str}</span>')
                i += run_len
                matched_repeat = True
                break
        if not matched_repeat:
            chunk_end = min(i + line_length, len(seq))
            chunk = seq[i:chunk_end]
            html_parts.append(chunk)
            i = chunk_end
        if i % line_length == 0 and i < len(seq):
            html_parts.append("\n")
    html_parts.append("</pre>")
    display(HTML("".join(html_parts)))

def display_colored_alignment(q_seq, s_seq, title="Colored Alignment"):
    html = [f"<h4>{title}</h4><pre style='background:#f8f8f8;padding:10px;font-family:monospace;'>"]
    step = 80
    for i in range(0, min(len(q_seq), len(s_seq)), step):
        q = q_seq[i:i+step]
        s = s_seq[i:i+step]
        line = ""
        for a, b in zip(q, s):
            if a == b and a != "-":
                line += "<span style='color:green;background:lightgreen'>|</span>"
            elif a == "-" or b == "-":
                line += "<span style='color:red'>-</span>"
            else:
                line += " "
        html += [f"Query : {q}<br>", f"        {line}<br>", f"Subject: {s}<br><br>"]
    html.append("</pre>")
    display(HTML("".join(html)))

# --- 3. Callback ---
def on_run_button_clicked(b):
    with output:
        clear_output()
        species_list = [s.strip() for s in species_input.value.split(",") if s.strip()]
        if len(species_list) < 2:
            print("❌ Enter at least 2 species.")
            return

        gene_name = gene_dropdown.value
        method = tree_method_dropdown.value
        feature = feature_dropdown.value
        total_species = len(species_list)

        print(f"\n🔬 Analyzing {', '.join(species_list)} (Gene: {gene_name}) – {feature} for Universal Panacea...")

        # DNA/RNA Section
        if feature in ["All Features", "DNA/RNA Sequence Only", "BLAST Comparisons (DNA/RNA)", "Phylogenetic Tree (DNA/RNA)", "Repeat Analysis", "Colored Alignments"]:
            print("\n--- DNA/RNA Sequences & Analysis ---")
            dna_sequences, dna_notes = fetch_dna_sequences(species_list, gene_name)
            for species in species_list:
                record = dna_sequences.get(species)
                note = dna_notes.get(species, "")
                if record is not None:
                    display_fasta(record, f"DNA/RNA for {species}")
                    analysis = analyze_dna(record)
                    print(f"{species}: {analysis['Length (bp)']} bp, {analysis['GC Content (%)']}% GC")
                    print(f"Taxonomy: {fetch_taxonomy(species)}")
                    img_url = fetch_google_image(species)
                    if img_url:
                        print(f"Image for {species}: {img_url}")
                        display(Image(url=img_url, width=200))
                    else:
                        print(f"Image not found for {species}")
                    if note:
                        print(f"Note: {note}")
                else:
                    print(f"{species}: {note}")
                print()  # Spacer

            # BLAST (DNA/RNA)
            if feature in ["All Features", "BLAST Comparisons (DNA/RNA)"]:
                print("\n--- DNA/RNA BLAST Comparisons ---")
                dna_comp_df = perform_blast_comparisons(dna_sequences, "DNA")
                if not dna_comp_df.empty:
                    display(dna_comp_df.style.hide(axis="index"))

            # Tree (DNA/RNA)
            if feature in ["All Features", "Phylogenetic Tree (DNA/RNA)"]:
                try:
                    print("\n--- Phylogenetic Tree (DNA/RNA) ---")
                    names, matrix = compute_distance_matrix(dna_sequences)
                    build_and_plot_tree(names, matrix, method, f"DNA/RNA Phylogenetic Tree: {', '.join(species_list)} ({gene_name})")
                    print("✅ DNA/RNA tree generated")
                except Exception as e:
                    print(f"❌ DNA/RNA tree error: {e}")

            # Repeats (DNA/RNA)
            if feature in ["All Features", "Repeat Analysis"]:
                print("\n--- DNA/RNA Repeat Analysis (Homopolymers >=3 nt) ---")
                all_repeats = {species: find_tandem_repeats(str(record.seq)) for species, record in dna_sequences.items() if record is not None}
                motif_counts = {}
                for species_repeats in all_repeats.values():
                    for char, start, end, length in species_repeats:
                        motif = (char, length)
                        motif_counts[motif] = motif_counts.get(motif, 0) + 1

                shared_motifs = {motif: count for motif, count in motif_counts.items() if count >= 2}
                if shared_motifs:
                    print(f"✅ Shared repeat motifs (in ≥2 species): {shared_motifs}")
                else:
                    print("ℹ️ No shared repeat motifs found.")

                for species in species_list:
                    record = dna_sequences.get(species)
                    if record is None:
                        continue
                    repeats = all_repeats.get(species, [])
                    if repeats:
                        df_data = [
                            {
                                "Nucleotide": char,
                                "Start Pos": start,
                                "End Pos": end - 1,
                                "Length": length,
                                "Shared in Species": motif_counts.get((char, length), 1)
                            }
                            for char, start, end, length in repeats
                        ]
                        df = pd.DataFrame(df_data)
                        print(f"\nRepeats table for {species} (with sharing analysis):")
                        display(df.style.hide(axis="index"))
                    else:
                        print(f"No repeats found in {species}.")
                    display_colored_seq(record, repeats, motif_counts, total_species)

                  # Protein Section
        if feature in ["All Features", "Protein Sequence Only", "BLAST Comparisons (Protein)", "Phylogenetic Tree (Protein)", "Colored Alignments"]:
            print("\n--- Protein Sequences & Analysis ---")
            protein_sequences, protein_notes = fetch_protein_sequences(species_list)  # Unpack both dicts
            for species in species_list:
                record = protein_sequences.get(species)
                note = protein_notes.get(species, "")
                if record is not None:
                    display_fasta(record, f"Protein for {species}")
                    analysis = analyze_protein(record)
                    print(f"{species}: {analysis['Length (aa)']} aa")
                    print(f"Taxonomy: {fetch_taxonomy(species)}")
                    img_url = fetch_google_image(species)
                    if img_url:
                        print(f"Image for {species}: {img_url}")
                        display(Image(url=img_url, width=200))
                    else:
                        print(f"Image not found for {species}")
                    if note:
                        print(f"Note: {note}")
                else:
                    print(f"{species}: {note}")
                print()

            # BLAST (Protein)
            if feature in ["All Features", "BLAST Comparisons (Protein)"]:
                print("\n--- Protein BLAST Comparisons ---")
                prot_comp_df = perform_blast_comparisons(protein_sequences, "Protein")
                if not prot_comp_df.empty:
                    display(prot_comp_df.style.hide(axis="index"))

            # Tree (Protein)
            if feature in ["All Features", "Phylogenetic Tree (Protein)"]:
                try:
                    print("\n--- Phylogenetic Tree (Protein) ---")
                    names, matrix = compute_distance_matrix(protein_sequences)
                    build_and_plot_tree(names, matrix, method, f"Protein Phylogenetic Tree: {', '.join(species_list)}")
                    print("✅ Protein tree generated")
                except Exception as e:
                    print(f"❌ Protein tree error: {e}")

            # Colored Alignments (Protein)
            if feature in ["All Features", "Colored Alignments"] and len(species_list) >= 2:
                print("\n--- Colored Protein Sequence Matching/Alignment ---")
                valid_protein = [(sp, rec) for sp, rec in protein_sequences.items() if rec is not None]
                if len(valid_protein) >= 2:
                    q_seq = str(valid_protein[0][1].seq)[:500]
                    s_seq = str(valid_protein[1][1].seq)[:500]
                    display_colored_alignment(q_seq, s_seq, f"Protein Alignment: {valid_protein[0][0]} vs {valid_protein[1][0]}")

        print(f"\nAnalysis complete! [Data sourced from NCBI/GenBank – Live as of {time.strftime('%Y-%m-%d %H:%M:%S')} IST]")

# Link button
run_button.on_click(on_run_button_clicked)